# 1. Get OpenAI API Key

Prior to fine-tune our model, let's get the OpenAI credentials needed for the API calls.

Go to [OpenAI website](https://platform.openai.com/api-keys) and create a new secrete key.

# 2. Create training data

The next step is to create training data to teach GPT-3 what you'd like to say. The data need to be a JSONL document with a new prompt and the ideal generated text:

```
{"prompt": "<question>", "completion": "<ideal answer>"}
{"prompt": "<question>", "completion": "<ideal answer>"}
{"prompt": "<question>", "completion": "<ideal answer>"}
```


**Optional for Colab users**

Before starting, we can set up the connection with the Google Drive storage, to keep there our documents.
Just execute the following passages:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Make sure that the variable path contains the correct sequence of folders separate by a `'/'` to get to your desired files

In [ ]:
import os

path = '_NLP/Project'

os.chdir(f'/content/drive/MyDrive/{path}')
os.getcwd()

'/content/drive/MyDrive/_NLP/Project'

Let's start by importing the libraries needed:

In [ ]:
!pip uninstall -y openai
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00


In [ ]:
import json
import openai
import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split

Then add your API key from the previous step:

In [ ]:
api_key ="sk-proj-###" # ADD YOUR API KEY HERE
openai.api_key = api_key

Now create a regular dict with the training data:

Load the dataset:

In [ ]:
dataset = load_dataset('arrow', data_files='data-00000-of-00001.arrow')
df = dataset['train'].to_pandas()
df.head()

Generating train split: 0 examples [00:00, ? examples/s]

,input,output,instruction
0,What is the relationship between very low Mg2+...,Very low Mg2+ levels correspond to low PTH lev...,Answer this question truthfully
1,What leads to genitourinary syndrome of menopa...,Low estradiol production leads to genitourinar...,Answer this question truthfully
2,What does low REM sleep latency and experienci...,Low REM sleep latency and experiencing halluci...,Answer this question truthfully
3,What are some possible causes of low PTH and h...,"PTH-independent hypercalcemia, which can be ca...",Answer this question truthfully
4,How does the level of anti-müllerian hormone r...,The level of anti-müllerian hormone is directl...,Answer this question truthfully


In [ ]:
df = df.iloc[:, :-1]

In [ ]:
len(df)

33955

In [ ]:
df.head()

,input,output
0,What is the relationship between very low Mg2+...,Very low Mg2+ levels correspond to low PTH lev...
1,What leads to genitourinary syndrome of menopa...,Low estradiol production leads to genitourinar...
2,What does low REM sleep latency and experienci...,Low REM sleep latency and experiencing halluci...
3,What are some possible causes of low PTH and h...,"PTH-independent hypercalcemia, which can be ca..."
4,How does the level of anti-müllerian hormone r...,The level of anti-müllerian hormone is directl...


In [ ]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_data.head()

,input,output
17426,What is the reason for the rapid-onset of acti...,What is the reason for the rapid-onset of acti...
21416,What types of cancer are associated with a dec...,Oral contraceptives are associated with a decr...
27343,What is the product of the conversion of galac...,Galactose is converted to galactose-1-phosphat...
985,What is the effect of prostaglandin agonists o...,Prostaglandin agonists increase the uveosclera...
13243,To which drug class do ipratropium and tiotrop...,Ipratropium and tiotropium belong to the drug ...


In [ ]:
# Initialize an empty list to store the training data
training_data = []

DEFAULT_SYSTEM_PROMPT = 'Answer this question truthfully.'

def create_dataset(question, answer):
    return {
        "messages": [
            {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer},
        ]
    }

# Iterate through the rows of the DataFrame
for index, row in train_data.iterrows():
    # Create a dictionary for each row
    data_entry = create_dataset(row["input"], row["output"])

    # Add the dictionary to the list
    training_data.append(data_entry)

In [ ]:
file_name = "turbo_training_data.jsonl"

with open(file_name, "w") as output_file:
 for entry in training_data:
  json.dump(entry, output_file)
  output_file.write("\n")

This file was used to fine tune the model using the OpenAI API UI, like this:

![OPENAI-UI](images/ft-ui.png)

And the result was:

![Results-FT-gpt-3.5-turbo](images/ft:gpt3.5-turbo-0125.png)